
# Staffing a Rural License Branch

This notebook walks through building a small discrete‑event simulation (DES) of a rural license branch. We'll:

1. Describe the system and translate the narrative into a process flow.
2. Define input probability distributions.
3. Build a simulation engine (event driven) in Python.
4. Run a baseline for one 10‑hour day (600 minutes) with 120 customers on average.
5. Explore where to add 1 more staff member (grader, driving examiner, or photo station).
6. Do simple input‑distribution sensitivity checks.




## 1. Problem Formulation

- The branch is open 10 hours/day = 600 minutes.
- Average 120 people/day → about 1 person every 5 minutes on average.
- Process:
  1. Arrival / check‑in (1 clerk): `Triangular(1, 3, 5)` minutes.
  2. Written exam (room has infinite seats): `Exponential(mean=25)` minutes.
  3. Grading (limited graders): exactly 3 minutes per exam.
  4. If pass (75%):  
     - 25% of those → must take driving test: `Normal(50, 15)` minutes, truncate at 0.1 minute.
     - 75% of those → skip driving.
  5. Photo station (1 server): `Triangular(0.5, 1, 1.5)` minutes.
     - After each photo there's a 10% chance the customer says “I don't like it” → they rejoin the photo queue.
  6. All movement between stations = 0.5 minutes (30 seconds).
  7. Written exam fail rate = 25% → those leave and come back another day.
  8. Driving test fail rate = 10% → those leave w/o license.
  9. All queues are first‑come first‑served.

**Goal:** help the branch decide where 1 extra person helps the most.


In [22]:

import math
import random
from dataclasses import dataclass, field
import heapq
from statistics import mean

random.seed(42)


## 2. Input distributions

In [23]:

def tri(a, m, b):
    """Triangular(a, m, b)."""
    return random.triangular(a, b, m)

def expo(mean):
    return random.expovariate(1 / mean)

def normal(mean, sd, min_val=0.1):
    x = random.gauss(mean, sd)
    return max(x, min_val)

# Baseline parameters (easier to tweak later)
ARRIVAL_MEAN = 600 / 120   # 5 minutes between arrivals on average
CHECKIN = (1.0, 3.0, 5.0)
WRITTEN_MEAN = 25.0
GRADING_TIME = 3.0
DRIVING = (50.0, 15.0)
PHOTO = (0.5, 1.0, 1.5)
MOVE_TIME = 0.5


## 3. Event‑driven simulation core

Define the Event and the Simulator

In [14]:

@dataclass(order=True)
class Event:
    time: float
    priority: int
    action: callable = field(compare=False)

class Simulator:
    def __init__(self, until=600):
        self.t = 0.0 # current simulation clock
        self.events = [] # priority queue of future events
        self.until = until # time to stop simulation
        self.logs = [] # optional list to record activity

    def schedule(self, time, action, priority=0):
        # Schedule new events
        heapq.heappush(self.events, Event(time, priority, action))

    def run(self):
        # Running the Simulation
        while self.events and self.t <= self.until:
            ev = heapq.heappop(self.events)
            self.t = ev.time
            ev.action(self)


In practice, each Event is something like:
“At time = 37.5, call the function that starts photo service for Customer #12.”

### 3.1 Simple FCFS resource

This class models a shared facility or staff member in the simulation — for example, a check-in clerk, an exam grader, or a photo station.
Each Resource has a limited number of servers (capacity), keeps track of who’s busy, and manages a first-come, first-served (FCFS) queue.

In [24]:

class Resource:
    def __init__(self, name, capacity=1):
        self.name = name
        self.capacity = capacity
        self.busy = 0
        self.queue = []  # (time_entered_queue, customer)

    def request(self, sim: Simulator, cust, start_service_cb):
        if self.busy < self.capacity:
            self.busy += 1
            start_service_cb(sim, cust)
        else:
            self.queue.append((sim.t, cust))

    def release(self, sim: Simulator, service_done_cb=None):
        self.busy -= 1
        if self.queue:
            _, cust = self.queue.pop(0)
            self.busy += 1
            if service_done_cb:
                service_done_cb(sim, cust)


## 4. Model the branch
Define customer class. Each customer has the following information.

**Id, Arrival time, photo-taking, written, driving**


In [25]:

class Customer:
    _id = 0
    def __init__(self, arrival_time):
        Customer._id += 1
        self.id = Customer._id
        self.arrival = arrival_time
        self.start_photo = None
        self.finished = None
        self.written_pass = None
        self.driving_needed = None

def build_system(extra=None):
    # biuld the resource and capacity

    resources = {
        "checkin": Resource("checkin", capacity=1),
        "grader": Resource("grader", capacity=1),
        "driving": Resource("driving", capacity=1),
        "photo": Resource("photo", capacity=1),
    }
    if extra == "grader":
        resources["grader"].capacity += 1
    elif extra == "photo":
        resources["photo"].capacity += 1
    elif extra == "driving":
        resources["driving"].capacity += 1
    return resources


### 4.1 Process logic
We define how the flow is processed.

In [26]:

def customer_flow(sim: Simulator, resources, cust: Customer, stats: dict):

    # c denotes a customer

    def start_checkin(sim, c):
        duration = tri(*CHECKIN) 
        sim.schedule(sim.t + duration, lambda sim: finish_checkin(sim, c))

    def finish_checkin(sim, c):
        resources["checkin"].release(sim, start_checkin)
        sim.schedule(sim.t + MOVE_TIME, lambda sim: start_exam(sim, c))

    resources["checkin"].request(sim, cust, start_checkin)

    def start_exam(sim, c):
        duration = expo(WRITTEN_MEAN)
        sim.schedule(sim.t + duration, lambda sim: exam_done(sim, c))

    def exam_done(sim, c):
        sim.schedule(sim.t + MOVE_TIME, lambda sim: grade_exam_request(sim, c))

    def grade_exam_request(sim, c):
        resources["grader"].request(sim, c, start_grading)

    def start_grading(sim, c):
        sim.schedule(sim.t + GRADING_TIME, lambda sim: finish_grading(sim, c))

    def finish_grading(sim, c):
        resources["grader"].release(sim, start_grading)
        if random.random() < 0.25:
            stats["written_fail"].append(sim.t - c.arrival)
            c.finished = sim.t
            return
        c.written_pass = True
        if random.random() < 0.25:
            c.driving_needed = True
            sim.schedule(sim.t + MOVE_TIME, lambda sim: request_driving(sim, c))
        else:
            sim.schedule(sim.t + MOVE_TIME, lambda sim: request_photo(sim, c))

    def request_driving(sim, c):
        resources["driving"].request(sim, c, start_driving)

    def start_driving(sim, c):
        duration = normal(*DRIVING)
        sim.schedule(sim.t + duration, lambda sim: finish_driving(sim, c))

    def finish_driving(sim, c):
        resources["driving"].release(sim, start_driving)
        if random.random() < 0.10:
            stats["driving_fail"].append(sim.t - c.arrival)
            c.finished = sim.t
            return
        sim.schedule(sim.t + MOVE_TIME, lambda sim: request_photo(sim, c))

    def request_photo(sim, c):
        resources["photo"].request(sim, c, start_photo)

    def start_photo(sim, c):
        if c.start_photo is None:
            c.start_photo = sim.t
        duration = tri(*PHOTO)
        sim.schedule(sim.t + duration, lambda sim: finish_photo(sim, c))

    def finish_photo(sim, c):
        if random.random() < 0.10:
            resources["photo"].release(sim, start_photo)
            sim.schedule(sim.t + MOVE_TIME, lambda sim: request_photo(sim, c))
            return
        resources["photo"].release(sim, start_photo)
        c.finished = sim.t
        stats["licensed"].append(c.finished - c.arrival)


### 4.2 Arrival generator and running one day

In [27]:

def run_day(
    extra=None,
    arrival_mean=ARRIVAL_MEAN,
    checkin=CHECKIN,
    exam_mean=WRITTEN_MEAN,
    driving_params=DRIVING,
    photo_params=PHOTO,
    seed=42,
):
    random.seed(seed)
    global CHECKIN, WRITTEN_MEAN, DRIVING, PHOTO
    CHECKIN = checkin
    WRITTEN_MEAN = exam_mean
    DRIVING = driving_params
    PHOTO = photo_params

    sim = Simulator(until=600)
    resources = build_system(extra=extra)
    stats = {
        "licensed": [],
        "written_fail": [],
        "driving_fail": [],
    }

    def make_arrivals(sim: Simulator):
        t = 0.0
        while t < sim.until:
            inter = expo(arrival_mean)
            t += inter
            c = Customer(arrival_time=t)
            sim.schedule(t, lambda sim, cust=c: customer_flow(sim, resources, cust, stats))

    make_arrivals(sim)
    sim.run()
    return stats, resources


## 5. Baseline run

In [28]:

stats, res = run_day()
print("Licensed:", len(stats["licensed"]))
print("Written fails (leave):", len(stats["written_fail"]))
print("Driving fails (leave):", len(stats["driving_fail"]))
if stats["licensed"]:
    print("Avg time in system (licensed):", round(mean(stats["licensed"]), 2), "min")


Licensed: 71
Written fails (leave): 36
Driving fails (leave): 1
Avg time in system (licensed): 58.26 min


## 6. Where to add?

In [29]:

def experiment_extras():
    configs = [None, "grader", "driving", "photo"]
    results = {}
    for cfg in configs:
        all_times = []
        for rep in range(100):
            s, _ = run_day(extra=cfg, seed=rep + 1)
            all_times += s["licensed"]
        results[cfg or "baseline"] = {
            "n_licensed": len(all_times),
            "avg_time": round(mean(all_times), 2) if all_times else None,
        }
    return results

results = experiment_extras()
results


{'baseline': {'n_licensed': 7292, 'avg_time': 57.41},
 'grader': {'n_licensed': 7256, 'avg_time': 54.29},
 'driving': {'n_licensed': 7899, 'avg_time': 51.95},
 'photo': {'n_licensed': 7306, 'avg_time': 56.39}}

## 7. Sensitivity to input distributions

In [21]:

def sensitivity():
    base, _ = run_day(seed=2)
    base_avg = mean(base["licensed"])

    scenarios = {}

    # 1) Busier day: arrivals every 4 minutes on avg
    s1, _ = run_day(arrival_mean=4, seed=1)
    scenarios["higher_arrival"] = mean(s1["licensed"])

    # 2) Slower checkin
    s2, _ = run_day(checkin=(2, 4, 6), seed=1)
    scenarios["slower_checkin"] = mean(s2["licensed"])

    # 3) Longer exams
    s3, _ = run_day(exam_mean=35, seed=1)
    scenarios["longer_exam"] = mean(s3["licensed"])

    # 4) Longer driving tests
    s4, _ = run_day(driving_params=(60, 20), seed=1)
    scenarios["longer_driving"] = mean(s4["licensed"])

    # 5) Slower photo
    s5, _ = run_day(photo_params=(1, 1.5, 2), seed=1)
    scenarios["slower_photo"] = mean(s5["licensed"])

    return base_avg, scenarios

base_avg, scen = sensitivity()
print("Baseline avg time (licensed):", round(base_avg, 2))
for k, v in scen.items():
    print(k, "→", round(v, 2))


Baseline avg time (licensed): 54.87
higher_arrival → 58.3
slower_checkin → 54.39
longer_exam → 51.51
longer_driving → 49.98
slower_photo → 70.74
